# Data Cleaning Sub-project

### Goal: Create a API cleaning pipeline

  This project is to create an API cleaning ETL pipeline that allows a given API to be cleaned based on the following criteria:

  * Handling Missing Data
  * Dealing with Duplicate Data
  * Standardization of Data
  * Handling Outliers of Data
  * Reshaping Data 
  * Filtering and Selecting Data

This data pipeline will also attempt to fit best practices in ETL pipeline building which means the following:

- Error handling
- Logging
- Testing

The pipeline will be built with an attempt to be able to scale however due to the nature of the project will be unneeded and will be an attempt at best practice

## Data Imports
All imports are listed below as well as versions used.

Current versions are:

In [ ]:
import logging 
import pandas as pd
import requests
import 